# SARSA (Estado - Acción - Recompensa - Estado - Acción)

Este tercer día está marchando genial, estamos cargados de energía y con muchas ganas de seguir aprendiendo, así que vamos a por el segundo algoritmo de **Aprendizaje por Reforzamiento**, que es uno llamado **SARSA**, y este curioso nombre proviene de las siglas en inglés de la expresión *"Estado-Acción-Recompensa-Estado-Acción"*.

Comencemos a clarificar un poco este concepto. **SARSA** es un algoritmo que también se utiliza para aprender la política de un *agente*. Recuerda que cuando decimos la *mejor política*, estamos diciendo la *mejor acción a tomar* en cada *estado*.

Esto es bastante similar a lo que hicimos con **Q-learning**, pero con la diferencia de mientras que Q-learning solo considera el *estado actual* del agente para tomar la siguiente decisión, **SARSA** toma en cuenta tanto el *estado actual* como la *acción actual* para predecir la próxima acción y su recompensa. Este enfoque se conoce como aprendizaje *"on-policy"*, donde el agente aprende el valor de la política que está siguiendo actualmente, incluyendo cómo decidirá sus acciones futuras.

La actualización de los **valores Q** en **SARSA** sigue la fórmula:

![](SARSA.png)

Esta ecuación podría leerse de esta manera:

> "El nuevo valor Q de un par estado-acción, es igual al valor Q anterior de (s, a), más el producto de multiplicar la tasa de aprendizaje alfa por la diferencia entre la recompensa observada r más el producto del factor de descuento gamma y el valor Q del siguiente estado y acción, Q de (s' , a'), menos el valor Q anterior de (s, a)."

¿Te vas a aprender eso de memoria? **Por supuesto que NO**. Yo tampoco. La idea, como siempre, es abrir el capot del auto durante un segundo para ver cómo es el motor, y ahora lo cerramos de inmediato, para sentarnos en el asiento del conductor y conducir nuestro coche usando el tablero de control que nos permitirá hacer funcionar ese motor de manera indirecta.

A continuación, verás las librerías que usaremos en esta lección. Nada nuevo bajo el sol. 

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

### Recursos para el código: variables y funciones


Vamos a hacer un ejercicio similar al de la lección anterior, donde tendremos una **cuadrícula de 4x4**, en la que esta vez no pondremos obstáculos.

Entonces vamos a proceder a definir todas las variables que necesitaremos para este ejercicio.

In [2]:
dimensiones = (4, 4)
estado_inicial = (0, 0)
estado_objetivo = (3, 3)
acciones = [(0, -1), (0, 1), (-1, 0), (1, 0)]
acciones_simbolos = ['↑', '↓', '←', '→']

Lo que sigue, es inicializar la **tabla Q**.

In [3]:
num_estados = dimensiones[0] * dimensiones[1]
num_estados

16

In [4]:
num_acciones = len(acciones)
num_acciones

4

In [5]:
Q = np.zeros((num_estados, num_acciones))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

Nos falta definir los **parametros SARSA**:

In [6]:
alpha = 0.1
gamma = 0.99
epsilon = 0.2
episodios = 1000

+ **`alpha = 0.1`**: Determina cuánto de la nueva información tiene que sobrescribir a la información antigua. Si ponemos una tasa de aprendizaje más alta permitimos que el agente ajuste sus estimaciones de **valor Q** más rápidamente como respuesta a la nueva información. Sin embargo, la desventaja de un valor demasiado alto es que puede hacer que el aprendizaje sea inestable. Entonces, un valor de 0.1 es un punto de partida común, que nos proporciona un equilibrio entre la estabilidad y la velocidad del aprendizaje.
+ **`gamma = 0-99`**: Indica la importancia que van a tener las recompensas futuras. Mientras más cercano a `1` sea el valor, hace que el agente valore más las recompensas futuras y que actúe más estratégicamente en el largo plazo. Entonces un valor de 0.99 es bastante común en entornos en los que deseamos que el agente tenga en cuenta el futuro cercano de manera más significativa.
+ **`epsilon = 0.2`**: Regula el equilibrio que tiene que haber entre la exploración de nuevas acciones y la explotación del conocimiento que ya ha sido adquirido. Poniendo un `epsilon` de `0.2`, tendremos un **20%** de probabilidad de que el agente elija una acción al azar. Esto fomenta que el agente explore el espacio de acción y evita que se quede atascado en una política que sea inferior a la política óptima pendiente de ser descubierta.
+ **`episodios = 1000`**: Define cuántas veces el agente va a repetir el entrenamiento completo. Mientras más alto sea el número de `episodios`, el agente tendrá más oportunidades para aprender de diferentes situaciones. En este caso yo considero que `1000` es un número suficientemente grande para permitir que el agente explore y aprenda adecuadamente en muchos entornos.

Ahora que todas las variables están definidas, comencemos a definir las **funciones** que va a necesitar nuestro algoritmo.

Primero vamos a necesitar una variable que se encargue de convertir el **estado* en un **índice lineal**. Recuerda que el estado consiste en *dos dimensiones*, pero el algoritmo va a necesitar ubicarlo en un *índice único* en nuestra nueva **tabla Q**.

In [7]:
def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

In [8]:
estado_a_indice((3, 0))

12

La segunda función auxiliar se va a encargar de decidir si la siguiente acción va a ser por explotación o por exploración.

In [9]:
def elegir_accion(estado):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, num_acciones - 1)
    else:
        return np.argmax(Q[estado_a_indice(estado)])

Y ahora que tenemos una función que **elige** la siguiente acción, vamos a definir una función que se encargue de **aplicar** esa acción.

In [10]:
def aplicar_accion(estado, accion_idx):
    accion = acciones[accion_idx]
    nuevo_estado = tuple(np.add(estado, accion) % np.array(dimensiones))
    
    if nuevo_estado == estado_objetivo:
        recompensa = 1
    else:
        recompensa = -1
    
    return nuevo_estado, recompensa, nuevo_estado == estado_objetivo

Antes de avanzar quiero explicar cuál es el sentido de la variable `nuevo_estado`. Vamos a analizar su contenido.

+ **`np.add(estado, accion)`**: Esta función de **NumPy** suma el *estado actual* y la *accion* que se va a tomar. Si *estado* es una **tupla** con la posición actual del agente en la cuadrícula (por ejemplo `(x, y)`) y *accion* es una **tupla** que representa el cambio en la posición debido a la acción (por ejemplo `(-1, 0)` para moverse hacia arriba), esta operación da como resultado la nueva posición del agente.  
+ **`% np.array(dimensiones)`**: Este operador de **módulo** asegura que si el agente intenta moverse más allá de los límites de la cuadrícula, la posición se "envuelve" al otro lado. ¿Cómo es esto? La función `np.array(dimensiones)` convierte las dimensiones de la cuadrícula en un **array de NumPy** para que la operación de módulo se aplique elemento por elemento. Es una forma de aplicar límites "atravesables", donde al cruzar un borde de la tabla se aparece por el lado opuesto (como en los juegos clásicos de "Asteroids" o el "Pac-Man").
+ **`tuple()`**: Convierte el resultado de vuelta en una **tupla**, ya que `np.add()` devuelve un **array de NumPy** y las operaciones subsiguientes también producen un array.

En resumen, esta línea actualiza la *posición del agente* (basándose en la acción que ha decidido tomar) y se asegura que la nueva posición se mantenga dentro de los límites del entorno.

Ten en cuenta que la decisión de que el agente pueda el comportamiento Pac-Maan de salir por la derecha y aparecer por la izquierda, es una decisión que yo he tomado para este ejemplo, pero se podría preferir algo distinto.


### Entrenamiento del modelo

Y ahora sí, finalmente tenemos todos los recursos que necesitamos (variables y funciones), y podemos comenzar a entrenar nuestro modelo.

In [11]:
for episodio in range(episodios):
    estado = estado_inicial
    accion_idx = elegir_accion(estado)
    terminado = False
    
    while not terminado:
        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)
        nueva_accion_idx = elegir_accion(nuevo_estado)
        
        indice = estado_a_indice(estado)
        Q[indice, accion_idx] += alpha * (recompensa + gamma * Q[estado_a_indice(nuevo_estado), nueva_accion_idx] - Q[indice, accion_idx])
        
        estado, accion_idx = nuevo_estado, nueva_accion_idx

Con esto, nuestro modelo ha realizado 1000 rondas de entrenamiento, por lo que podemos estar seguros de que ha desarrollado todo el aprendizaje necesario para ser capaz de identificar la política más óptima.


### Visualizar los resultados

La última etapa de nuestro código va a consistir en la implementación de una **visualización**, para que podemos ver cuál es la política que ha descubierto nuestro agente.

In [12]:
politica_simbolos = np.empty(dimensiones, dtype='<U2')
politica_simbolos

array([['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', '']], dtype='<U2')

Ahí tenemos la matriz en la que vamos a visualizar los símbolos de los movimientos con las flechas, pero por ahora está vacía.

In [13]:
for i in range(dimensiones[0]):
    for j in range(dimensiones[1]):
        estado = (i, j)
        mejor_accion = np.argmax(Q[estado_a_indice(estado)])
        politica_simbolos[i, j] = acciones_simbolos[mejor_accion]

politica_simbolos

array([['←', '←', '↓', '←'],
       ['↑', '↓', '↑', '←'],
       ['↑', '↑', '→', '→'],
       ['↑', '↓', '↓', '↑']], dtype='<U2')

Gracias a esta acción ya tenemos la matriz rellenada con el símbolo que corresponda al mejor movimiento para cada estado. 

Espero que hayas disfrutado de esta lección. Hemos aprendido nuestro segundo algoritmo de **Aprendizaje por Reforzamiento**, pero el día aún no se termina porque hay más algoritmos que aprender, y te recomiendo hacerlo ya mismo.